In [1]:
# jupyter imports
from IPython.display import clear_output


In [2]:
import pandas as pd

In [3]:
FILE_TO_ADD_PATH =  '/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_standart_low_hwe/results/both_phen.csv'
FILE_TO_CREATE_PATH = '/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_standart_low_hwe/results/both_phen_whole_genome_fixed_with_genes_-e-5.csv'
GENE_FILE_PATH = '/cs/labs/michall/roeizucker/virt_env_install_test/genes_hg19.csv'
WORK_ONLY_FOR_RES_VARIANTS = True
FILTER_PVAL = True
FILTER = 10**-5

In [4]:
CHR_SYMBOL = "#CHROM"
POS_SYMBOL = "POS"
PVAL_SYMBOL = "P"

In [5]:
def find_gene_heler(tree,row):
    if tree is None:
        return None
    if row["POS"] <= tree.val["cds_end"] and row["POS"] >= tree.val["cds_start"]:
        return tree.val
    if row["POS"] <= tree.val["cds_start"]:
        return find_gene_heler(tree.left,row)
    return find_gene_heler(tree.right,row)
    
def find_gene(genes_dict,row):
    return find_gene_heler((genes_dict[str(row["#CHROM"])]),row)
    pass

def add_genes_for_snps_symbol(df,genes_csv):
    rows = []
    for index,row in df.iterrows():
        if index % 10000 == 0:
            clear_output(wait=True)
            print(str(int(index/len(df) * 100)) + "%")

        val = row
        gene = find_gene(genes_dict,row)
        rows.append([row,gene])
    return rows
    

In [6]:
# create gene tree
class TreeNode:
    def __init__(self,val=None,left=None,right=None):
        self.val = val
        self.left = left
        self.right = right

def add_to_node(node,val):
    if val["cds_start"] < node.val["cds_start"]:
        if node.left is None:
            node.left = TreeNode(val)
        else:
            add_to_node(node.left,val)
    else:
        if node.right is None:
            node.right = TreeNode(val)
        else:
            add_to_node(node.right,val)
    pass
genes_df = pd.read_csv(GENE_FILE_PATH)
genes_dict = {}
for index, row in genes_df.iterrows():
    if row['chr'] not in genes_dict:
        genes_dict[row['chr']] = TreeNode(row)
    else:
        add_to_node(genes_dict[row['chr']],row)



In [7]:
# genes_dict

In [8]:
# print(find_gene(genes_dict,first_row))

In [9]:
# test_csv = "/cs/labs/michall/roeizucker/IIH/GWAS/results/iih_mod_small_cov.csv"
# test_df = pd.read_csv(test_csv)
# first_row = None
# for val in test_df.iterrows():
#     first_row = val[1]
#     if find_gene(genes_dict,first_row) is not None:
#         print(find_gene(genes_dict,first_row) ,val)
#         print("#"*30)
    
# first_row

In [10]:
# add_genes_for_snps_symbol(df,genes_df)


In [11]:
df = pd.read_csv(FILE_TO_ADD_PATH)
# df["length"] = df["length"].astype(int)
# df.to_csv(FILE_TO_CREATE_PATH)

In [12]:
if WORK_ONLY_FOR_RES_VARIANTS:
    df = df[df[PVAL_SYMBOL].notnull()]
if FILTER_PVAL:
    df = df[df["P"] < FILTER]
vals = add_genes_for_snps_symbol(df,genes_df)


In [13]:
index = 0
final_results = []
for val in vals:
    if index % 10000 == 0:
        clear_output(wait=True)
        print(str(int(index/len(df) * 100)) + "%")
    index+=1    
    new_0 = val[0].to_dict()
    if val[1] is None:
#         val[0]["gene_symbol"] = "no_gene_found"
#         val[0]["length"] = -1
#         val[0]["name"] = "no_gene_found"
#         final_results.append(val[0])
        new_0["gene_symbol"] = "no_gene_found"
        new_0["length"] = -1
        new_0["name"] = "no_gene_found"
        final_results.append(new_0)

        continue
    new_1 = val[1].to_dict()
    new_0["gene_symbol"] = new_1["symbol"]
    new_0["length"] =  int(new_1["cds_end"]) -  int(new_1["cds_start"])
    new_0["name"] = new_1["name"]
    final_results.append(new_0)
    

0%


In [14]:
len(final_results)

1109

In [15]:
new_df = pd.DataFrame.from_records(final_results)

In [16]:
new_df.to_csv(FILE_TO_CREATE_PATH)

In [17]:
new_df[new_df["length"] > 0]

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE,gene_symbol,length,name
1,1,7090989,rs537339895,G,T,Y,T,G,0.000853,N,ADD,295550,12.95350,0.507840,5.04365,4.567460e-07,.,CAMTA1,980957,calmodulin binding transcription activator 1
5,1,22203106,rs137904249,C,T,Y,T,C,0.001758,N,ADD,296013,7.50798,0.453078,4.44949,8.607280e-06,.,HSPG2,113898,heparan sulfate proteoglycan 2
7,1,32194480,rs148253895,A,C,Y,C,A,0.001031,N,ADD,295477,10.65780,0.507451,4.66309,3.114980e-06,.,ADGRB2,29798,adhesion G protein-coupled receptor B2
8,1,33986908,1:33986908_TG_T,TG,T,Y,T,TG,0.006362,N,ADD,294868,4.77591,0.308251,5.07244,3.927400e-07,.,CSMD2,646293,CUB and Sushi multiple domains 2
9,1,34002283,rs192836088,C,A,Y,A,C,0.006398,N,ADD,295262,4.31481,0.321286,4.55063,5.348490e-06,.,CSMD2,646293,CUB and Sushi multiple domains 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,22,28498158,rs186457733,A,C,Y,C,A,0.003514,N,ADD,295943,5.48841,0.381531,4.46265,8.095170e-06,.,TTC28,697499,tetratricopeptide repeat domain 28
1100,22,32917276,rs534497302,G,A,Y,A,G,0.001731,N,ADD,293537,7.89458,0.449944,4.59207,4.388670e-06,.,SYN3,492965,synapsin III
1101,22,34060972,rs111694724,C,T,Y,T,C,0.003239,N,ADD,294405,5.82041,0.382352,4.60667,4.091680e-06,.,LARGE1,487047,LARGE xylosyl- and glucuronyltransferase 1
1102,22,34095499,rs2283930,G,A,Y,A,G,0.212479,N,ADD,296013,1.63970,0.111624,4.43014,9.417040e-06,.,LARGE1,487047,LARGE xylosyl- and glucuronyltransferase 1


In [18]:
df[df["P"] < 0.00005]

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE
24991,1,6095424,rs537531827,T,C,Y,C,T,0.001786,N,ADD,295585,7.60499,0.452897,4.47961,7.477960e-06,.
28993,1,7090989,rs537339895,G,T,Y,T,G,0.000853,N,ADD,295550,12.95350,0.507840,5.04365,4.567460e-07,.
51503,1,12915645,rs576880983,C,T,Y,T,C,0.003181,N,ADD,284231,6.10679,0.381872,4.73824,2.155860e-06,.
53162,1,13987288,rs184931912,G,A,Y,A,G,0.001159,N,ADD,295860,9.40301,0.503460,4.45125,8.537000e-06,.
72591,1,18295369,rs7412790,A,G,Y,G,A,0.001138,N,ADD,294711,9.76460,0.507221,4.49264,7.034450e-06,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11018305,22,38714156,rs78870725,G,A,Y,A,G,0.007712,N,ADD,294855,3.90822,0.306356,4.44934,8.613370e-06,.
11023844,22,40234943,rs546694816,T,A,Y,A,T,0.001066,N,ADD,295593,10.18950,0.507332,4.57561,4.748380e-06,.
11056944,22,47851009,rs185659053,A,G,Y,G,A,0.003962,N,ADD,295331,5.45269,0.360034,4.71097,2.465400e-06,.
11058045,22,48074516,rs192540303,G,A,Y,A,G,0.004213,N,ADD,294058,5.11880,0.359373,4.54380,5.524950e-06,.


In [19]:
# df = df.drop("GENE",axis='columns')
# df.to_csv(FILE_TO_CREATE_PATH)

In [20]:
genes_df[(genes_df["chr"] == "1") & (genes_df["cds_start"] < 11253684) & (genes_df["cds_end"] > 11253684)]

,Unnamed: 0,uniprot_id,symbol,name,refseq_ids,chr,cds_start,cds_end
1467,1467,O43827,ANGPTL7,angiopoietin like 7,['NM_021146'],1,11249637,11255077
4715,4715,P42345,MTOR,mechanistic target of rapamycin kinase,['NM_004958'],1,11167545,11319466


In [21]:
FILE_TO_CREATE_PATH

'/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_standart_low_hwe/results/both_phen_whole_genome_fixed_with_genes_-e-5.csv'

In [22]:
new_df

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE,gene_symbol,length,name
0,1,6095424,rs537531827,T,C,Y,C,T,0.001786,N,ADD,295585,7.60499,0.452897,4.47961,7.477960e-06,.,no_gene_found,-1,no_gene_found
1,1,7090989,rs537339895,G,T,Y,T,G,0.000853,N,ADD,295550,12.95350,0.507840,5.04365,4.567460e-07,.,CAMTA1,980957,calmodulin binding transcription activator 1
2,1,12915645,rs576880983,C,T,Y,T,C,0.003181,N,ADD,284231,6.10679,0.381872,4.73824,2.155860e-06,.,no_gene_found,-1,no_gene_found
3,1,13987288,rs184931912,G,A,Y,A,G,0.001159,N,ADD,295860,9.40301,0.503460,4.45125,8.537000e-06,.,no_gene_found,-1,no_gene_found
4,1,18295369,rs7412790,A,G,Y,G,A,0.001138,N,ADD,294711,9.76460,0.507221,4.49264,7.034450e-06,.,no_gene_found,-1,no_gene_found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,22,38714156,rs78870725,G,A,Y,A,G,0.007712,N,ADD,294855,3.90822,0.306356,4.44934,8.613370e-06,.,no_gene_found,-1,no_gene_found
1105,22,40234943,rs546694816,T,A,Y,A,T,0.001066,N,ADD,295593,10.18950,0.507332,4.57561,4.748380e-06,.,ENTHD1,144065,ENTH domain containing 1
1106,22,47851009,rs185659053,A,G,Y,G,A,0.003962,N,ADD,295331,5.45269,0.360034,4.71097,2.465400e-06,.,no_gene_found,-1,no_gene_found
1107,22,48074516,rs192540303,G,A,Y,A,G,0.004213,N,ADD,294058,5.11880,0.359373,4.54380,5.524950e-06,.,no_gene_found,-1,no_gene_found
